In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import glob
import os
import random

import psutil
import random
import time
import sys
import math
from contextlib import contextmanager
#reference: https://www.kaggle.com/c/indoor-location-navigation/discussion/215445

In [ ]:
feature_dir = "../input/indoor-navigation-and-location-wifi-features/wifi_features"

In [ ]:
#wifi features in format of rows = measurements and 
#cols = wifi signal from each access point

In [ ]:
predictions = list()

data = pd.read_csv(train_files[0], index_col=0)

x_train = data.iloc[:,:-4]
y_trainy = data.iloc[:,-3]
y_trainx = data.iloc[:,-4]
y_trainf = data.iloc[:,-2]

In [ ]:
from sklearn.model_selection import KFold
N_SPLITS = 20
SEED = 1234

kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
for fold, (trainid, testid) in enumerate(kf.split(data.iloc[:, :-4])):
    random.seed(1234)
    np.random.seed(1234)
    X_train = data.iloc[trainid, :-4]
    y_trainx = data.iloc[trainid, -4]
    y_trainy = data.iloc[trainid, -3]

    X_valid = data.iloc[testid, :-4]
    y_validx = data.iloc[testid, -4]
    y_validy = data.iloc[testid, -3]

    modelx = lgb.LGBMRegressor(objective='root_mean_squared_error',boosting_type='gbdt',
                               num_leaves=90,n_estimators=50000,learning_rate=0.1)
    modelx.fit(X_train, y_trainx, eval_set=[(X_valid, y_validx)], eval_metric='rmse',
               early_stopping_rounds=5)

    modely = lgb.LGBMRegressor(objective='root_mean_squared_error',boosting_type='gbdt',
                               num_leaves=90,n_estimators=50000,learning_rate=0.1)
    modely.fit(X_train, y_trainy, eval_set=[(X_valid, y_validy)], eval_metric='rmse',
               early_stopping_rounds=5)



In [ ]:
#testing different numbers of leaves and estimators
evaldata = []
for i in [100, 115, 125, 127, 150]:
    for n in [50,60,75,90,100]:
        modely = lgb.LGBMRegressor(n_estimators=n, num_leaves=i, n_jobs=1)
        gkf = sk.model_selection.GroupKFold()
        scores = sk.model_selection.cross_val_score(
           modely, x_train, y_trainy, scoring='neg_mean_squared_error',
           cv=gkf, groups=data.iloc[:,-1], n_jobs=5)

        evaldata.append([i, n, scores.mean()] )